In [1]:
exec(open("init_notebook.py", "r").read())

current working dir/home/abjawad/Documents/GitHub/local-attention-model


In [2]:
from models.swin_transformer import PatchEmbed
from models.encoders.student_segformer import OverlapPatchEmbed, Mlp, DWConv, Block, RGBXTransformer
from models.decoders.MLPDecoder import DecoderHead
from models.encoders.merge_attn import MultiScaleAttention
from functools import partial
import torch.nn as nn
import torch

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


In [3]:
swin_patch_embed = PatchEmbed(img_size=224, 
                              patch_size=4, 
                              in_chans=3, 
                              embed_dim=96, 
                              norm_layer=None)

overlap_patch_embed = OverlapPatchEmbed(patch_size=7, 
                                        stride=4, 
                                        in_chans=3, 
                                        embed_dim=64)

mlp = Mlp(in_features=64,
          hidden_features=64*4,
          act_layer=torch.nn.GELU,
          drop=0)

multiscale_attn = MultiScaleAttention(dim=64,
                                      num_heads=2,
                                      sr_ratio=8,
                                      local_region_shape=[8, 16],
                                      img_size=(256, 256)) 

block = Block(dim=64,
              num_heads=2, 
              mlp_ratio=4,
              qkv_bias=True,
              qk_scale=None,
              drop=0,
              attn_drop=0, 
              drop_path=0,
              norm_layer=torch.nn.LayerNorm,
              sr_ratio=8,
              local_region_shape=[8, 16],
              img_size=(256, 256))


rgbx_transformer = RGBXTransformer(img_size=(1024, 1024), patch_size=4, 
                                   embed_dims=[32, 64, 160, 256], 
                                   num_heads=[2, 4, 5, 8], 
                                   mlp_ratios=[4, 4, 4, 4], 
                                   qkv_bias=True, 
                                   norm_layer=partial(nn.LayerNorm, eps=1e-6), 
                                   depths=[2,2,2,2], 
                                   sr_ratios=[8, 4, 2, 1], 
                                   drop_rate=0.0, drop_path_rate=0.1)
# in_channels=self.channels, num_classes=cfg.num_classes, norm_layer=norm_layer, embed_dim=cfg.decoder_embed_dim
mlp_decoder = DecoderHead(in_channels =  [64, 128, 320, 512],
                         num_classes = 19,
                         norm_layer = nn.BatchNorm2d,
                         embed_dim = 512)

In [4]:
# # craete random input and pass through the model
# x_1 = torch.rand(1, 3, 224, 224)
# y_1 = swin_patch_embed(x_1)

# x_2 = torch.rand(1, 3, 1024, 1024)
# y_2 = overlap_patch_embed(x_2)

# x_3, h, w = torch.rand(1, 65536, 64), 256, 256
# y_3 = mlp(x_3, h, w)

# x_4 = torch.rand(1, 65536, 64)
# y_4 = multiscale_attn(x_4, 256, 256)

# x_5 = torch.rand(1, 65536, 64)
# y_5 = block(x_5, 256, 256)

x_6 = torch.rand(1, 3, 1024, 1024)
y_6 = rgbx_transformer(x_6)

# torch.Size([1, 64, 256, 256]) torch.Size([1, 128, 128, 128]) torch.Size([1, 320, 64, 64]) torch.Size([1, 512, 32, 32])
x_7 = [torch.rand(1, 64, 256, 256), torch.rand(1, 128, 128, 128), torch.rand(1, 320, 64, 64), torch.rand(1, 512, 32, 32)]
y_7 = mlp_decoder(x_7)

# print(swin_patch_embed.flops())
# print("overlap_patch_embed ", overlap_patch_embed.flops())
# print("MLP ", mlp.flops())
# print("multiscale_attn ", multiscale_attn.flops())
# print(block.flops())
rgbx_flops = rgbx_transformer.flops()
mlp_flops = mlp_decoder.flops()
# print(rgbx_transformer.flops())
# print(mlp_decoder.flops())

print("total ", rgbx_flops + mlp_flops)

input:  torch.Size([1, 3, 1024, 1024])
x before proj:  torch.Size([1, 3, 1024, 1024])
x after proj:  torch.Size([1, 32, 256, 256])
x after norm: !!!!!!!!!!!!!!!!!!  torch.Size([1, 65536, 32])
############### Stage 1 ##########################
tokenization:  torch.Size([1, 65536, 32])
+++++++++ block +++++ input:  torch.Size([1, 65536, 32]) 256 256
!!!!!!!!!!!!attention head:  2  !!!!!!!!!!
torch.Size([1, 65536, 32]) 256 256
input: MLP  torch.Size([1, 65536, 32]) 256 256
+++++++++ block +++++ input:  torch.Size([1, 65536, 32]) 256 256
!!!!!!!!!!!!attention head:  2  !!!!!!!!!!
torch.Size([1, 65536, 32]) 256 256
input: MLP  torch.Size([1, 65536, 32]) 256 256
output:  torch.Size([1, 32, 256, 256])
******** End Stage 1 **************
############### Stage 2 ##########################
x before proj:  torch.Size([1, 32, 256, 256])
x after proj:  torch.Size([1, 64, 128, 128])
x after norm: !!!!!!!!!!!!!!!!!!  torch.Size([1, 16384, 64])
tokenization:  torch.Size([1, 16384, 64])
+++++++++ block

##### self.patch_embed1 = OverlapPatchEmbed(patch_size=7, stride=4, in_chans=in_chans, embed_dim=64)

##### input:  torch.Size([1, 3, 1024, 1024])
##### ############### Stage 1 ##########################
##### tokenization:  torch.Size([1, 65536, 64])
##### output:  torch.Size([1, 64, 256, 256])
##### ******** End Stage 1 **************
##### ############### Stage 2 ##########################
##### :  torch.Size([1, 16384, 128])
##### output:  torch.Size([1, 128, 128, 128])
##### ******** End Stage 2 **************
##### ############### Stage 3 ##########################
##### tokenization:  torch.Size([1, 4096, 320])
##### output:  torch.Size([1, 320, 64, 64])
##### ******** End Stage 3 **************
##### ############### Stage 4 ##########################
##### tokenization:  torch.Size([1, 1024, 512])
##### output:  torch.Size([1, 512, 32, 32])
##### ******** End Stage 4 **************